# <center>Supervised Learning - Text Classification</center>
References:
* http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

## 1. Finally, we come to machine learning ...
<img src='https://res.cloudinary.com/practicaldev/image/fetch/s--_jRAhLTB--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://thepracticaldev.s3.amazonaws.com/i/7brl707yigrhno91vc6d.jpg' width = "50%">

* Review basic concepts of machine learning
  * Cross validation
  * Performance metrics: recall and precision, AUC, PRC
* Text Classification  
  * Assign a document into one  or more pre-defined categories (or labels)
  * **Single-label**: e.g. spam dection, sentiment detection
  * **Multi-label**: e.g. news categorization

## 2. Review basic concepts of machine learning

### 2.1. Model assessment and selection - How valid is a model? 

- Generalization: the prediction capability of a model ($f$) on independent test data, 
    - Given testing samples ($X, Y$), and prediction $\hat{Y} = f(X, \theta)$), where $\theta$ is the model (hyper) parameters, e.g. $K$ in KNN 
    - Prediction error (or `loss function`): $L(Y, \hat{Y})$, e.g.
        - Squared error: $L(Y, \hat{Y}) = (Y-\hat{Y})^2$
        - Absolute error: $L(Y, \hat{Y}) = |Y-\hat{Y}|$
  
- Data-rich situation: split data into training, validation, and test sets (e.g. 50%, 25%, 25%)
  - **Training** set: fit the model
  - **Validation** set (sometimes called evaluation set): 
     - tune model parameters
     - estimate prediction error for model selection
  - **Test** set: assess the prediction erorr of the final chosen model
     - in machine learning competition, usually a test set is given without truth labels
     
  <img src="train_validation_test.png" width="40%">


### 2.2. Cross Validation
- However, labeled data is always scarce. We cannot afford to set aside a validation set
- `K-fold cross validation`: 
    1. Data is separated into k subsets. Each time, one of the subsets is held as the validation set (a.k.a holdout, test set) and the rest of them is used as the training set. 
    <img src="cross_validation.png" width="40%"> [source] (http://spark-public.s3.amazonaws.com/nlp/slides/sentiment.pptx)
    2. This method repeats *k* times and each time with a different subset as the test set. 
    3. Calculate average prediction error `(CV)` on `K` validation sets $$ CV(\theta) = \frac{1}{N} \sum_{i=1}^{N}{L~(~y_i,~f^{k(i)}(x_i, \theta)~)}$$ where:
       - $\theta$: the model parameters (e.g. the number of neighbours in $k$-NN), 
       - $f^{k(i)}$: model fitted on the $k$th iteration, 
       - $N$: number of samples
    4. Tune model parameters $(~\theta~)$ to minize the average prediction error
       - How? e.g. How to determine $K$ in KNN?
    5. Select the model with the minimal prediction error (along with $\theta$ determined)
    6. Fit the selected model to all the data
  

### 2.3. Performance metrics and model evaluation
- **Accuracy** is often used to measure model performance
- $ Accurarcy = \frac{\#~correct~predictions}{\#~of~total~samples}$
- However, for an imbalanced classification problem where `one category represents the overwhelming majority of the data points`, accuracy can be a problematic metric
  - e.g. prediction of rare diseases: Assume only 1 patient in every 1M population. What is the accuracry if you predict everyone is healthy?
  - e.g. prediction of earthquakes

 #### 2.3.1 Precision, Recall, and F-score
  * Precision: precentage of true cases among the predicated true cases
  * Recall:  precentage of true cases that have been retrieved over the total number of true cases
  * F-score: $$\frac{2*precision*recall}{precision+recall}$$
  * Example: 
  
     Confusion Matrix: <img src="confusion_matrix.png" width="50%">
    * For "YES" group: 
      - precision=?, 
      - recall=?, 
      - f-score=?
    * For "NO" group:
      - precision=?, 
      - recall=?, 
      - f-score=?
  * Overall model performance
    * precision_macro (or recall_macro or f1_macro) is calculated as:
      1. calculate precision for each label
      2. average over labels 
    * precision_micro (or recall_micro or f1_micro): calculates metrics globally regardless of labels. 
      - precision_micro = recall_micro = accuracy
      - Note precision_micro and recall_micro are the same for single label classification
    * With inbalanced classes, the difference between these two metrics may be significant, e.g.
    - class 1: 950 samples, precision 90%
    - class 2: 50 samples, precision 70%

#### 2.3.2. True Positive `(TP)` Rate and False Positive `(FP)` Rate
- Two types of prediction errors:
  - False Positive: Predict a sample is positive when it is in fact negative.
  - False Negative: Predict a sample is negative when in fact it is positive.

<img src="classification_error.png" width="60%">

- Metrics: <br>
   $ 
    \begin{align}
     precision~of~positive~class &= \frac{true~positives}{ true~positives + false~positives} \\
     recall~of~positive~class &= \frac{true~positives}{ true~positives + false~negatives} \\
     true~positive~rate~(TPR) &= recall~of~positive~class  &= sensitivity\\
     false~positive~rate~(FPR) &= \frac{false~positives}{ false~positives + true~negatives} &= 1-Specificity\\
     & = 1- recall~of~negative~class
    \end{align}
    $
    - Note, $false~positive~rate \neq 1-true~positive~rate$
- Now calculate TPR and FPR of the the image below:
<img src="confusion_matrix.png" width="50%">
- For more details, read https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c

- Two types of predictions in classification problems:
   - Predict class labels directly, e.g. report the class of the majority of K nearest neighbors in KNN
   - Predict a **probability** (i.e. confidence) for each class, e.g. the percentage of neighbors of a class 
      - Calibrate a probability threshold to determine class labels for desired performance levels
      - How the threshold can affect precision, recall, true positive rate, and false positive rate? e.g. threshold 0.90 vs. 0.10
      
<img src = "threshold_table.png" width="60%">

- Example:
  - $threshold = 0.5$: $ precision = \frac{42}{42+16} =0.724$, $ recall = tpr = \frac{42}{42+13} = 0.76$, $ fpr = \frac{16}{29+16} = 0.36$
  - $threshold = 0.9 $: $ precision = \frac{12}{12+3} =0.8$, $ recall = tpr = \frac{12}{12+40} = 0.23$, $ fpr = \frac{3}{3+45} = 0.0625$
  - As the threshold increases, for the positive class, how does each of the metric change?
    - precision?
    - recall or true positive rate (tpr)?
    - false positive rate (fpr)?

<img src="all_metrics.png" width = "50%">


- Trade-offs:
  - Precision vs. recall (high prcision but low recall, and vice versa)
    -  As the threshold increases, precision increases but recall decreases
  - True positive rate vs. false positive rate 
    -  As the threshold increases, both tpr and fpr decrease. Ideally, we like to have high tpr but low fpr
  - In real classification problems, you may prefer one metric to the other
    - e.g. earthquake prediction
    - e.g. desease prediction

### 2.3.3 AUC and Precision-Recall Curve
- **Receiver operating characteristic (ROC) curve**: plots the true positive rate (TPR) vs the false positive rate (FPR) as a function of the model’s threshold for classifying positives
<img src="roc.png" width="40%">
- **Area under the curve (AUC)**: metric to calculate the overall performance of a classification model based on area under the ROC curve 
   - Which model is better?
   - How to determine best threshold?
  <img src="auc_curve.png" with="50%">
- **Precision-Recall Curve** : plot the precision (y-axis) and the recall (x-axis) for different thresholds, much like the ROC curve.
<img src="precision_recall_curve.png" width="50%">
  - Which algorithm is better?
<img src="precision_recall.png" width="50%">
 source: https://i.stack.imgur.com/T0kQr.png
 
- More about ROC and PRC: 
  - When the dataset is extremely imbalanced, i.e. the majority is negative samples, a model with high AUC is not necessarily good. For experiment, check https://www.kaggle.com/lct14558/imbalanced-data-why-you-should-not-use-roc-curve
  - For in-depth discussion between ROC and PRC, read Davis, J. and Goadrich, M., The relationship between Precision-Recall and ROC curves. http://pages.cs.wisc.edu/~jdavis/davisgoadrichcamera2.pdf
   

## 3. Text Classification

* Problem formulation:
    * Input: 
      - A document $d$ 
      - A fixed set of classes C = {$c_1$, $c_2$,..., $c_J$}
      - A training set of $m$ hand-labeled documents ($d_1,c_1$),....,($d_m,c_m$)
    * Output: a classifier that predicts $d$ to some classes $c$ $\subset$ C
* Basic process
  1. Load and preprocess sample data
  2. Extract features: e.g. bag of words with TF-IDF weights
  3. Split feature space into trainning and test sets following cross validation method
  4. Train a classifier/model with the training dataset using selected classification algorithm for each fold
  5. Calculate performance
 
* Considerations for deciding text classification algorithms
  - should be effective in high dimensional spaces (`curse of dimensionality`)
  - should be effective even if `the number of features is greater than the number of samples`
  - some good algorithms to start with:
      - Naive Bayes (https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf): baseline for performance benchmarking of text classification algorithms. **Read Pages 13-35 for a good understanding of naive Bayes algorithm**
      - Support Vector Machine (SVM). References:
        - http://www.robots.ox.ac.uk/~az/lectures/ml/lect2.pdf. **Read pages 2-4, 9, 12-19, and 29-38 for a good understanding of SVM**.
        - https://www.svm-tutorial.com/2014/11/svm-understanding-math-part-1/
  

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np

np.set_printoptions(precision=3)
                    
import pandas as pd

In [ ]:
# Exercise 3.1.: Load data 
# Load datasets (http://qwone.com/~jason/20Newsgroups/)
# For convenience, a subset of the data has been saved into "twenty_news_data.csv"

import pandas as pd
data=pd.read_csv("../../dataset/twenty_news_data.csv",header=0)
data.head()

# print out the full text of the first sample
print(data["text"][0])

## 3.1. TF-IDF matrix generation
- Function: **sklearn.feature_extraction.text.TfidfVectorizer**(input='content',encoding='utf-8', decode_error='strict', token_pattern='(?u)\b\w\w+\b', lowercase=True, stop_words=None, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, norm='l2', use_idf=True, smooth_idf=True, ...)
- Some useful parameters:
    * **input** : string {'filename', 'file', 'content').
    * **token_pattern** : Regular expression denoting what constitutes a “token”. The default is '(?u)\b\w\w+\b', i.e. a token contains at least two word characters in unicode (note: ?u: unicode, \b: space or non-word character, i.e. boundary, \w: word character). 
    * **ngram_range** : tuple (min_n, max_n): The lower and upper boundary of the range of n-values for different n-grams to be extracted. 
    * **stop_words** : string {‘english’}, list, or None (default)
    * **lowercase** : boolean, default True: Convert all characters to lowercase before tokenizing.
    * **max_df/min_df** : float in range [0.0, 1.0] or int, default=1.0: When building the vocabulary ignore terms that have a document frequency strictly higher (lower) than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. 
    * **max_features** : int or None, default=None. If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
    * **norm** : 'l1', 'l2' or None, optional. Norm used to normalize term vectors. None for no normalization.
    * **use_idf** : boolean, default=True. Enable inverse-document-frequency reweighting.
    * **smooth_idf** : boolean, default=True. Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.
    * **binary** : If True, all non-zero term counts are set to 1. This does not mean outputs will have only 0/1 values, only that the tf term in tf-idf is binary. (Set idf and normalization to False to get 0/1 outputs).
- For all the parameters, see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [ ]:
# Exercise 3.2 Create TF-IDF Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the TfidfVectorizer 

tfidf_vect = TfidfVectorizer() 

# with stop words removed
#tfidf_vect = TfidfVectorizer(stop_words="english") 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(data["text"])

print("type of dtm:", type(dtm))
print("size of tfidf matrix:", dtm.shape)


In [ ]:
# Exercise 3.3. Examine TF-IDF

# 1. Check vocabulary

# Vocabulary is a dictionary mapping a word to an index

# the number of words in the vocabulary
print("total number of words:", len(tfidf_vect.vocabulary_))

print("type of vocabulary:", \
      type(tfidf_vect.vocabulary_))
print("index of word 'city' in vocabulary:", \
      tfidf_vect.vocabulary_['city'])



In [ ]:
# 3.4 check words with top tf-idf wights in a document, 
# e.g. 1st document

# get mapping from word index to word
# i.e. reversal mapping of tfidf_vect.vocabulary_
voc_lookup={tfidf_vect.vocabulary_[word]:word \
            for word in tfidf_vect.vocabulary_}

print("\nOriginal text: \n"+data["text"][0])

print("\ntfidf weights: \n")

# first, covert the sparse matrix row to a dense array
doc0=dtm[0].toarray()[0]
print("Vectorized document shape: ", doc0.shape, "\n")

# get index of top 20 words
print("top words:")
top_words=(doc0.argsort())[::-1][0:20]
for i in top_words:
    print("{0}:\t{1:.3f}".format(voc_lookup[i], doc0[i]))
#[(voc_lookup[i], '%.3f'%doc0[i]) for i in top_words]



In [ ]:
# Exercise 3.5. classification using a single fold

# use MultinomialNB algorithm
from sklearn.naive_bayes import MultinomialNB

# import method for split train/test data set
from sklearn.model_selection import train_test_split

# import method to calculate metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

# split dataset into train (70%) and test sets (30%)
X_train, X_test, y_train, y_test = train_test_split(\
                dtm, data["label"], test_size=0.3,\
                    random_state=0)

# train a multinomial naive Bayes model using the testing data
clf = MultinomialNB().fit(X_train, y_train)

# predict the news group for the test dataset
predicted=clf.predict(X_test)

# check a few samples
predicted[0:3]
y_test[0:3]

In [ ]:
# Exercise 3.6. Performance evaluation: 
# precision, recall, f1-score

# get the list of unique labels
labels=sorted(data["label"].unique())

# calculate performance metrics. 
# Support is the number of occurrences of each label

precision, recall, fscore, support=\
     precision_recall_fscore_support(\
     y_test, predicted, labels=labels)

print("labels: ", labels)
print("precision: ", precision)
print("recall: ", recall)
print("f-score: ", fscore)
print("support: ", support)

# another way to get all performance metrics
print(classification_report\
      (y_test, predicted, target_names=labels))

In [ ]:
# Exercise 3.7.  AUC 

from sklearn.metrics import roc_curve, auc,precision_recall_curve
import numpy as np

# We need to get probabilities as predictions
predict_p=clf.predict_proba(X_test)

# a probability is generated for each label
labels
predict_p[0:3]
# Ground-truth
y_test[0:3]

# let's just look at one label "soc.religion.christian"
# convert to binary
binary_y = np.where(y_test=="soc.religion.christian",1,0)

# this label corresponds to last column
y_pred = predict_p[:,3]

# compute fpr/tpr by different thresholds
# positive class has label "1"
fpr, tpr, thresholds = roc_curve(binary_y, y_pred, \
                                 pos_label=1)
# calculate auc
print("AUC: {:.2%}".format(auc(fpr, tpr)))

In [ ]:
from matplotlib import pyplot as plt
plt.figure();
plt.plot(fpr, tpr, color='darkorange', lw=2);
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--');
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('False Positive Rate');
plt.ylabel('True Positive Rate');
plt.title('AUC of Naive Bayes Model');
plt.show();

In [ ]:
# Exercise 3.8.  precision_recall_curve

# compute precision/recall by different thresholds
precision, recall, thresholds = precision_recall_curve(binary_y, \
                                y_pred, pos_label=1)

plt.figure();
plt.plot(recall, precision, color='darkorange', lw=2);
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('Recall');
plt.ylabel('Precision');
plt.title('Precision_Recall_Curve of Naive Bayes Model');
plt.show();

# Calculate area under PRC curver (a.k.a average precision)
# calculate auc
print("Average Precision: {:.2%}".format(auc(recall, precision)))

In [ ]:
# Exercise 3.9.  predict new documents

docs_new = ['God is love', 'OpenGL on the GPU is fast']

# generate tifid for new documents
# note we use "transform" not "fit_transform"
# transform creates tfidf vectors based on the
# vocabulary established by "fit_transform" in Exercise 3.2.
X_new_tfidf = tfidf_vect.transform(docs_new)

print("new sample tf_idf size:", X_new_tfidf.shape)

# prediction
clf.predict(X_new_tfidf)
clf.predict_proba(X_new_tfidf)

In [ ]:
# Exercise 3.10. Classification with stop words removed
# Can removing stop words improves performance?
# In Exercise 3.2, uncomment line 10 and comment line 7
# Run Exercise 3.2, 3.5-3.9

In [ ]:
# Exercise 3.10. Run 5-fold cross validation
# to show the generalizability of the model

# import cross validation method
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB

metrics = ['precision_macro', 'recall_macro', \
           "f1_macro"]

clf = MultinomialNB()
#clf = MultinomialNB(alpha=0.5)

cv = cross_validate(clf, dtm, data["label"], \
                    scoring=metrics, cv=5, \
                    return_train_score=True)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average f1 score:")
print(cv['test_f1_macro'])

# To see the performance of training data set use 
# cv['train_xx_macro']
print("\nTraining data average f1 score:")
print(cv['train_f1_macro'])

# The metrics are quite stable across folds.
# The performance gap between training and test sets is small
# This indicates the model has good generalizability

In [ ]:
# Exercise 3.11. Multinominal NB 
# with different smoothing parameter alpha
# comment line 11 and uncomment 12 in Exercise 3.8
# use different alpha value to see if it affects performance

In [ ]:
# Exercise 3.12. SVM model

from sklearn.model_selection import cross_validate
#from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm

metrics = ['precision_macro', 'recall_macro', "f1_macro"]

# initiate an linear SVM model
clf = svm.LinearSVC()

cv = cross_validate(clf, dtm, data["label"], \
                    scoring=metrics, cv=5)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average fscore:")
print(cv['test_f1_macro'])




## 3.3. Parameter tuning using grid search
* Each classification model has a few parameters
  * e.g. "stop_words": "english" or None, min_df: [1,2,3, ...]
  * e.g. MultinomialNB(alpha=1.0)
  * e.g. LinearSVC(C=1.0, penalty=’l2’, loss=’squared_hinge’,...)
* Instead of tweaking the parameters of the various components, it is possible to run an exhaustive search of the best parameters on a grid of possible values

In [ ]:
# Exercise 3.3.1 Grid search

# import pipeline class
from sklearn.pipeline import Pipeline

# import GridSearch
from sklearn.model_selection import GridSearchCV

# build a pipeline which does two steps all together:
# (1) generate tfidf, and (2) train classifier
# each step is named, i.e. "tfidf", "clf"

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', svm.LinearSVC())
                   ])

# set the range of parameters to be tuned
# each parameter is defined as 
# <step name>__<parameter name in step>
# e.g. min_df is a parameter of TfidfVectorizer()
# "tfidf" is the name for TfidfVectorizer()
# therefore, 'tfidf__min_df' is the parameter in grid search

parameters = {'tfidf__min_df':[1,3],
              'tfidf__stop_words':[None,"english"],
              'clf__C': [0.5,1.0,5.0],
}

# the metric used to select the best parameters
metric =  "f1_macro"

# GridSearch also uses cross validation
gs_clf = GridSearchCV\
(text_clf, param_grid=parameters, \
 scoring=metric, cv=5)

# due to data volume and large parameter combinations
# it may take long time to search for optimal parameter combination
# you can use a subset of data to test
gs_clf = gs_clf.fit(data["text"], data["label"])


In [ ]:
# gs_clf.best_params_ returns a dictionary 
# with parameter and its best value as an entry

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name,\
                                 gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))


In [ ]:
# Exercise 3.3.2 Grid search
# Modify Exercise 3.3 and Exercise 3.8 
# to use the best parameters found
# re-create the Multinominal NB classifier

# also, when setting min_df to 2, check the size of  
# your tf-idf feature matrix. 
# How much of the dimension is reduced?

## 4. Multi-label classification
- So far we only cover single-label classification, i.e. assign one class to each sample
- Multilabel classification emerges as a challenging problem, where classes are not mutually exclusive 
  * music categorization 
  * semantic classification of images
  * tagging
- **One-Vs-the-Rest** Strategy (a.k.a **one-vs-all**)
  * fitting one classifier per class. For each classifier, the class is fitted against all the other classes.
  * for $n$ classes (labels), $n$ classifier is needed
  * Advantage: good interpretability - Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier
  * Disadvantage: 
     * many classifiers are created if there is a large number classes
     * ignore the structure (or dependencies) of classes
- **Class indication matrix** (or **one-hot encoding**): Encode categorical integer features using a one-hot aka one-of-K scheme. 

| Document    | Money       | Investment | Crime & Justice |
| :-----------|:-----------:|:----------:|:--------------:|
| 1           | 0           |      0     | 1              |
| 2           | 1           |      1     | 0              |
| 3           | 1           |      0     | 0              |
| 4           | 0           |      1     | 1              |

- **dataset**: Yahoo News Ranked Multilabel Learning dataset (http://research.yahoo.com)
  - A subset is selected
  - 4 classes, 6426 samples
  
- **Discussion**: can you apply Naive Bayes for multi-label classification?

In [ ]:
# Exercise 4.1 Multi-label classification- Load data

import json
data=json.load(open("../../dataset/ydata.json","r"))

docs,labels=zip(*data)

# show sample examples
docs[1]
labels[1]


In [ ]:
# Exercise 4.2 One-hot coding of classes

from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(labels)
# check size of indicator matrix
# print some rows 
Y[0:5]
Y.shape
# check classes
mlb.classes_

# check # of samples in each class
np.sum(Y, axis=0)

In [ ]:

# Exercise 4.3 Multi-label classification- one vs. rest classifier

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# split dataset into train (70%) and test sets (30%)
X_train, X_test, Y_train, Y_test = train_test_split(\
                docs, Y, test_size=0.3, random_state=0)

classifier = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words="english",\
                              min_df=2)),
    ('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, Y_train)

In [ ]:
# Exercise 4.4 Multi-label classification- Performance report

from sklearn.metrics import classification_report

predicted = classifier.predict(X_test)

predicted.shape
print("predicted:")
predicted[0:2]
print("actual:")
Y_test[0:2]

print(classification_report\
      (Y_test, predicted, target_names=mlb.classes_))

In [1]:
import os

print(os.getcwd())

/Users/harshitahiremath/BIA 660-C : Web Mining
